In [1]:
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import random
import math
from sklearn import metrics
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
from pathlib import Path

In [6]:
df_test = pd.read_csv('/Users/yunzhengzhao/usf/summer/dl/project/data/test.csv')
df_test[:10000].to_csv('/Users/yunzhengzhao/usf/summer/dl/project/data/test1.csv', index=False)
df_test[10001:20000].to_csv('/Users/yunzhengzhao/usf/summer/dl/project/data/test2.csv', index=False)
df_test[20001:30000].to_csv('/Users/yunzhengzhao/usf/summer/dl/project/data/test3.csv', index=False)
df_test[30001:40000].to_csv('/Users/yunzhengzhao/usf/summer/dl/project/data/test4.csv', index=False)
df_test[40001:50000].to_csv('/Users/yunzhengzhao/usf/summer/dl/project/data/test5.csv', index=False)
df_test[50001:60000].to_csv('/Users/yunzhengzhao/usf/summer/dl/project/data/test6.csv', index=False)
df_test[60001:].to_csv('/Users/yunzhengzhao/usf/summer/dl/project/data/test7.csv', index=False)

In [2]:
raw = pd.read_csv('jigsaw-toxic-comment-train.csv')

FileNotFoundError: [Errno 2] File b'jigsaw-toxic-comment-train.csv' does not exist: b'jigsaw-toxic-comment-train.csv'

In [6]:
raw.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


## String cleaning functions

In [14]:
import numpy as np
from collections import defaultdict
import re

In [15]:
# this is from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()

In [16]:
def get_vocab(list_of_content):
    """Computes Dict of counts of words.
    
    Computes the number of times a word is on a document.
    """
    vocab = defaultdict(float)
    for content in list_of_content:
        for line in content:
            line = clean_str(line.strip())
            words = set(line.split())
            for word in words:
                vocab[word] += 1
    return vocab

In [7]:
raw_x = raw["comment_text"]

In [8]:
raw_y = raw["toxic"]

In [22]:
raw_x[0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [24]:
x = np.array([clean_str(line.strip()) for line in raw_x])

In [25]:
x.shape

(223549,)

In [26]:
y = np.array(raw_y)

In [27]:
y.shape

(223549,)

In [28]:
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [29]:
X_train[:5], y_train[:5]

(array(["above user has no en talk page , but does have one at de benutzer diskussion istiller i do n't see a problem with line breaks in the text rather , it 's some junction shapes which are missing the icons are pretty much all on commons , so any missing from en are likely missing from de also",
        "adelson thanks for the tip ! over at wp right we could really use your help please consider becoming a part of the fastest growing most influential ensemblages of editors in the entire wiki wp wikiproject conservatism about us lionel \\( talk \\) republican party presidential candidates , 2012 i for some reason still have this page on my watchilist , and see you 've been quite busy here lately i subsequently have had the page protected for a while hopefully that helps 1992 heh , thanks i 've been resisting the impulse to hit rfpp over the single intermittent but persistently disruptive ip i 've been \\( along with others \\) reverting , but get the feeling from other edits over con

In [30]:
word_count = get_vocab([X_train])

In [33]:
len(word_count.keys())

219847

In [34]:
for word in list(word_count):
    if word_count[word] < 5:
        del word_count[word]
len(word_count.keys())

39197

In [35]:
vocab2index = {"<PAD>":0, "UNK":1} # init with padding and unknown
words = ["<PAD>", "UNK"]
for word in word_count:
    vocab2index[word] = len(words)
    words.append(word)

## Encoding training and validation set

In [36]:
x_len = np.array([len(x.split()) for x in X_train])

In [37]:
np.percentile(x_len, 95) # let set the max sequence len to N=40

248.0

In [38]:
def encode_sentence(s, N=200):
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in s.split()])
    l = min(N, len(enc1))
    enc[:l] = enc1[:l]
    return enc

In [39]:
class ToxicityDataset(Dataset):
    def __init__(self, X, y):
        self.x = X
        self.y = y
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x = self.x[idx]
        x = encode_sentence(x)
        return x, self.y[idx]
    
train_ds = ToxicityDataset(X_train, y_train)
valid_ds = ToxicityDataset(X_val, y_val)

In [43]:
train_ds[2]

(array([ 93, 144, 143, 138,  81, 139,  23, 147,  93, 140, 150,   6, 152,
        137,   1,  93,  10,  44,  81,   6,   1, 142, 141,  23,  47, 149,
        153,  81,  17, 145,   9, 148,  93, 151,  17, 145,   9, 148,  69,
        146,  93,  10,  30, 139,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0

In [44]:
train_dl = DataLoader(train_ds, batch_size=100, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=100)

## 1D CNN model for sentence classification

Notation:
* V -- vocabulary size
* D -- embedding size
* N -- MAX Sentence length

In [55]:
class SentenceCNN(nn.Module):
    
    def __init__(self, V, D):
        super(SentenceCNN, self).__init__()
        self.embedding = nn.Embedding(V, D, padding_idx=0)

        self.conv_3 = nn.Conv1d(in_channels=D, out_channels=100, kernel_size=3)
        self.conv_4 = nn.Conv1d(in_channels=D, out_channels=100, kernel_size=4)
        self.conv_5 = nn.Conv1d(in_channels=D, out_channels=100, kernel_size=5)
        
        self.bn= nn.BatchNorm1d(1500)
        
        self.dropout = nn.Dropout(p=0.5)
        self.fc = nn.Linear(1500, 1)
        
    def forward(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        x = x.transpose(1,2)
        x3 = F.relu(self.conv_3(x))
        x4 = F.relu(self.conv_4(x))
        x5 = F.relu(self.conv_5(x))
        x3 = nn.MaxPool1d(kernel_size = 38)(x3)
        x4 = nn.MaxPool1d(kernel_size = 37)(x4)
        x5 = nn.MaxPool1d(kernel_size = 36)(x5)
        out = torch.cat([x3, x4, x5], 2)
        out = out.view(out.size(0), -1)
        out = self.dropout(self.bn(out))
        return self.fc(out)   

In [56]:
V = len(words)
D = 50
N = 40
model = SentenceCNN(V, D)

In [104]:
x, y = next(iter(train_dl))

In [105]:
y_hat = model(x.long().cuda())
y_hat.size()

torch.Size([100, 1])

In [106]:
y

tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0])

In [107]:
y_hat=y>0
sum(y_hat==y)

tensor(100)

In [97]:
F.binary_cross_entropy_with_logits(y_hat.cuda(), y.float().unsqueeze(1).cuda())

tensor(0.0377, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

## Training

In [74]:
def update_optimizer(optimizer, lr):
    for i, param_group in enumerate(optimizer.param_groups):
        param_group["lr"] = lr

In [75]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    
    acc = torch.round(acc) * 100
    
    return acc

In [90]:
def valid_metrics(model):
    model.eval()
    total = 0
    sum_loss = 0
    correct = 0
    for x, y in valid_dl:
        x = x.long().cuda()
        y = y.float().unsqueeze(1).cuda()
        batch = y.shape[0]
        out = model(x).cuda()
        loss = F.binary_cross_entropy_with_logits(out, y)
        sum_loss += batch*(loss.item())
        total += batch
        pred = (out > 0).float()
        correct += (pred == y).float().sum().item()
        #multi_acc(y_train_pred, y_train_batch)
    val_loss = sum_loss/total
    val_acc = correct/total
    return val_loss, val_acc

In [91]:
def train_epocs(model, optimizer, epochs=10):
    iterations = epochs*len(train_dl)
    pbar = tqdm_notebook(total=iterations)
    for i in range(epochs):
        model.train()
        total_loss = 0
        total = 0
        for x, y in train_dl:
            x = x.long().cuda()
            y = y.float().unsqueeze(1).cuda()
            out = model(x)
            loss = F.binary_cross_entropy_with_logits(out, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += x.size(0)*loss.item()
            total += x.size(0)
        train_loss = total_loss/total
        val_loss, val_accuracy = valid_metrics(model)
        
        print("train_loss %.3f val_loss %.3f val_accuracy %.3f" % (
            train_loss, val_loss, val_accuracy))

In [89]:
V = len(words)
D = 100
model = SentenceCNN(V, D).cuda()

In [92]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
train_epocs(model, optimizer, epochs=10)

train_loss 0.214 val_loss 0.153 val_accuracy 0.950
train_loss 0.140 val_loss 0.123 val_accuracy 0.954
train_loss 0.125 val_loss 0.132 val_accuracy 0.953
train_loss 0.118 val_loss 0.155 val_accuracy 0.950
train_loss 0.113 val_loss 0.145 val_accuracy 0.952
train_loss 0.106 val_loss 0.141 val_accuracy 0.953
train_loss 0.101 val_loss 0.145 val_accuracy 0.952
train_loss 0.094 val_loss 0.149 val_accuracy 0.951
train_loss 0.088 val_loss 0.176 val_accuracy 0.952
train_loss 0.083 val_loss 0.161 val_accuracy 0.952
